In [ ]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LinearRegression

from two_layer_net import TwoLayerNet


print('hello world')

In [ ]:
df_origin = pd.read_csv('open/train.csv')

train_df = pd.read_csv('open/train.csv')
test_df = pd.read_csv('open/test.csv')
sub_df = pd.read_csv('open/sample_submission.csv')

print('Number of Data for training : {col}\n Number of Variables : {row}'
      .format(col = train_df.shape[0], row = train_df.shape[1]))

In [ ]:
train_df.loc[[611,612,613,614,615]]

In [ ]:
# 2018.01.01 ~ 2020.12.31 (NaN of 2019.09.07 sat ;;;;;)
# 
new_row = pd.DataFrame(train_df.loc[[613]], columns = train_df.columns)

new_df = pd.concat([train_df.iloc[:613], new_row, train_df.iloc[613:]], ignore_index = True)

new_df.loc[614,'date'] = '2019-09-07' 


print(new_df.loc[[612,613,614,615]])
len(new_df)

In [ ]:
# To distinguish weekend and weekday
# The name of new index is "days of week" 
# [0:Mon , 1:Tue, 2:Wen, 3:Thur, 4:Fri, 5:Sat, 6:Sun]

df_week = [] # List for df column "days of week"

L1 = new_df.shape[0] // 7  # 1096 // 7
L2 = new_df.shape[0] % 7  # 1096 // 7

#week = [0, 1, 2, 3, 4, 5, 6]
week = [0, 0, 0, 0, 0, 1, 1] # weekday / weekend

for i in range(0,L1,1):
    df_week.extend(week)

for i in range(0,L2,1):
    df_week.append(0)

print(df_week)
print(len(df_week))

In [ ]:
year = []
month = []
day = [] 

for date in new_df["date"] :
    y = date.split('-')[0]
    m = date.split('-')[1]
    d = date.split('-')[2]
    
    year.append(y)
    month.append(m)
    day.append(d)

new_df["year"] = year 
new_df["month"] = month 
new_df["day"] = day

In [ ]:
new_df

In [ ]:
# Appended New Col "days of week"
new_df['days_of_week'] = df_week
new_df

In [ ]:
SS_list_Nan= new_df['sunshine_sum'][new_df['sunshine_sum'].isnull()].index
print(SS_list_Nan)

In [ ]:
new_df['sunshine_rate'].loc[SS_list_Nan].values.reshape(-1,1)

In [ ]:
new_df[['sunshine_rate', 'sunshine_sum']].corr(method='pearson') # Pearson Correlation

a = new_df['sunshine_sum'].dropna()
b = new_df['sunshine_rate'].drop(SS_list_Nan)
a.isnull().sum()
b.isnull().sum()

plt.plot(a,b, 'o')

In [ ]:
SS_list_Nan

In [ ]:
a = a.values.reshape(-1,1) # a is sum
b = b.values.reshape(-1,1) # b is rate

In [ ]:
print(a)
print(b)
new_df['sunshine_rate'].loc[SS_list_Nan] 

In [ ]:
line_fitter = LinearRegression()

line_fitter.fit(b, a) # predict a(sum) through b(rate) 
SS_sol_Nan = line_fitter.predict(new_df['sunshine_rate'].loc[SS_list_Nan].values.reshape(-1,1))
SS_sol_Nan = list(SS_sol_Nan)
float(SS_sol_Nan[1])

In [ ]:
new_df["PM10"] = new_df["PM10"].fillna(new_df["PM10"].mean())
new_df["PM2.5"] = new_df["PM2.5"].fillna(new_df["PM2.5"].mean())
new_df["sunshine_sum"] = new_df["sunshine_sum"].fillna(0)


print(new_df['sunshine_sum'].loc[SS_list_Nan])
print(float(SS_sol_Nan[1]))

for i, j in zip(SS_list_Nan, range(0,5,1)):
    new_df['sunshine_sum'].loc[i] = float(SS_sol_Nan[j])

In [ ]:
print(new_df['sunshine_sum'].loc[SS_list_Nan])

In [ ]:
PP_list_Nan= new_df['precipitation'][new_df['precipitation'].isnull()].index
print(PP_list_Nan)

pp = new_df['precipitation'].dropna()
srss = new_df[['sunshine_rate','sunshine_sum']].drop(PP_list_Nan)

In [ ]:
#pp = pp.values.reshape(-1,1) # pp is precipitation
#srss = srss.values.reshape(-1,1) # sr is sun_ratio

In [ ]:
line_fitter.fit(srss, pp)

In [ ]:
pp_sol_Nan = line_fitter.predict(new_df[['sunshine_rate','sunshine_sum']].loc[PP_list_Nan].values)
pp_sol_Nan = list(pp_sol_Nan)
pp_sol_Nan

In [ ]:
new_df.isnull().sum()

In [ ]:
print(test_df.isnull().sum())

In [ ]:
x = new_df.drop(['precipitation','date','rental'], axis=1)
y = new_df['rental']

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.25, shuffle=True)

test_x = test_df.drop(['date'], axis = 1)

In [ ]:
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [ ]:
reg = RandomForestRegressor()
reg.fit(train_x, train_y)
val_pred = reg.predict(val_x)

NMAE(val_y, val_pred)


In [ ]:
train_x

In [ ]:
train_y

In [ ]:
network = TwoLayerNet(input_size=14, hidden_size=5, output_size=2) #  Activation function : ReLU

iters_num = 10000
train_size = train_x.shape[0]
batch_size = 100
learning_rate = 0.15

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1) # max(8, 1)


In [ ]:
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = train_x[batch_mask]
    t_batch = train_y[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(train_x, train_y)
        test_acc = network.accuracy(train_x, train_y)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)